# The Battle of the Neighborhoods

## IBM Applied Data Science Capstone Project

### Jack Perng

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

    
1. [Introduction](#intro)

2. [Data](#data)

3. [Methodology](#method)

4. [Results](#results)

5. [Discussion](#discussion)

6. [Conclusion](#conclusion)
    
    
    
</font>
</div>

## 1. Introduction 
<a id = "intro"></a>

**Business Problem**: If someone just obtained their California acupuncture license and resided in San Jose, where would be a good location to setup their practice? <br>

The idea is to explore the San Jose neighborhoods, according to ZIP code, based on the follow factors that may be related to business growth:
1. Population<br>
2. Per capita income <br>
3. Existing competition <br>
4. Crime rate <br>
5. Unemployment rate <br>
6. Bachelor degree percentage <br>
7. Median home price <br>

The neighborhoods would be clustered via **k-means clustering**, analyzed and compared. The goal is to identify, via the clustered results, potential neighborhood candidates to start an acupuncture clinic in.


## 2. Data
<a id = "data"></a>

**Data Sources**: The data for the project would come from the various sources: 


1. Kaggle dataset "US Wages via Zipcode", containing the following information: <br>
a. U.S. ZIP codes <br>
b. Geographic coordinates <br>
c. Estimated Population <br>
d. Total Wages <br>
From this dataset, the relevant data subset for San Jose, CA can be extracted. In addition, the per capita income can be calculated from. <br> (https://www.kaggle.com/pavansanagapati/us-wages-via-zipcode)
2. Foursquare location data, which will be used to find and locate the acupuncturists in the San Jose area. The results are used as the metric for existing competition in the same ZIP code. 

3. ADT Security Services. The website contains an ADT Crime Map which provides the crime rate of each ZIP code. The "Total Crime" rate metric is selected. 

4. City-Data. The website provides the following data of interest:<br>
a. Unemployment rate <br>
b. Bachelor degree or higher percentage <br>
for population 25 and over.  

5. Zillow contains data on typical home prices based on ZIP code. The latest home prices at the time of completing this project, published on 05/31/2020, are used. 

The combined information of all the data described above provide the input to the clustering algorithm model.     

## 3. Methodology
<a id = "method"></a>

### 3.1 Kaggle Dataset

Import pandas

In [1]:
import pandas as pd

Read the Kaggle dataset into DataFrame

In [2]:
sj_zip_wages_df = pd.read_csv('free-zipcode-database-Primary.csv')
sj_zip_wages_df.head()

,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
0,705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,False,NaN,NaN,NaN
1,610,STANDARD,ANASCO,PR,PRIMARY,18.28,-67.14,NA-US-PR-ANASCO,False,NaN,NaN,NaN
2,611,PO BOX,ANGELES,PR,PRIMARY,18.28,-66.79,NA-US-PR-ANGELES,False,NaN,NaN,NaN
3,612,STANDARD,ARECIBO,PR,PRIMARY,18.45,-66.73,NA-US-PR-ARECIBO,False,NaN,NaN,NaN
4,601,STANDARD,ADJUNTAS,PR,PRIMARY,18.16,-66.72,NA-US-PR-ADJUNTAS,False,NaN,NaN,NaN


In [3]:
sj_zip_wages_df.shape

(42522, 12)

Clean up the Dataframe by selecting the relevant entries of San Jose, CA. Also remove the ZIP Codes that are PO Boxes. Lastly, double-check that the ZIP Code is not decommisioned. 

In [4]:
sj_zip_wages_df = sj_zip_wages_df[sj_zip_wages_df.City == 'SAN JOSE']
sj_zip_wages_df = sj_zip_wages_df[sj_zip_wages_df.State == 'CA']
sj_zip_wages_df = sj_zip_wages_df[sj_zip_wages_df.ZipCodeType != 'PO BOX']
sj_zip_wages_df = sj_zip_wages_df[sj_zip_wages_df.Decommisioned == False]

In [5]:
print(sj_zip_wages_df.shape)
sj_zip_wages_df.head()

(36, 12)


,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
23873,95101,STANDARD,SAN JOSE,CA,PRIMARY,37.38,-121.89,NA-US-CA-SAN JOSE,False,NaN,NaN,NaN
23878,95110,STANDARD,SAN JOSE,CA,PRIMARY,37.34,-121.90,NA-US-CA-SAN JOSE,False,7460.0,12621.0,366468568.0
23879,95111,STANDARD,SAN JOSE,CA,PRIMARY,37.28,-121.83,NA-US-CA-SAN JOSE,False,23302.0,43578.0,866020686.0
23880,95112,STANDARD,SAN JOSE,CA,PRIMARY,37.34,-121.88,NA-US-CA-SAN JOSE,False,21203.0,34111.0,891795651.0
23881,95113,STANDARD,SAN JOSE,CA,PRIMARY,37.33,-121.89,NA-US-CA-SAN JOSE,False,781.0,1049.0,37924110.0


Select the relevant columns

In [6]:
sj_zip_wages_df = sj_zip_wages_df[['Zipcode', 'Lat', 'Long', 'EstimatedPopulation', 'TotalWages']]
sj_zip_wages_df

,Zipcode,Lat,Long,EstimatedPopulation,TotalWages
23873,95101,37.38,-121.89,NaN,NaN
23878,95110,37.34,-121.90,12621.0,3.664686e+08
23879,95111,37.28,-121.83,43578.0,8.660207e+08
23880,95112,37.34,-121.88,34111.0,8.917957e+08
23881,95113,37.33,-121.89,1049.0,3.792411e+07
23883,95116,37.35,-121.85,35357.0,6.238882e+08
23884,95117,37.31,-121.96,22030.0,6.661914e+08
23885,95118,37.25,-121.88,26249.0,8.777164e+08
23886,95119,37.22,-121.78,8171.0,2.887622e+08
23887,95120,37.19,-121.83,33486.0,1.704123e+09


Lastly, remove the ZIP codes with missing values (NaN). Reset indices.

In [7]:
sj_zip_wages_df.dropna(axis = 0, inplace = True)
sj_zip_wages_df.reset_index(drop = True, inplace = True)
print(sj_zip_wages_df.shape)
sj_zip_wages_df.head()

(28, 5)


,Zipcode,Lat,Long,EstimatedPopulation,TotalWages
0,95110,37.34,-121.90,12621.0,366468568.0
1,95111,37.28,-121.83,43578.0,866020686.0
2,95112,37.34,-121.88,34111.0,891795651.0
3,95113,37.33,-121.89,1049.0,37924110.0
4,95116,37.35,-121.85,35357.0,623888214.0


Add "per capita wages" column by dividing the TotalWages by EstimatedPopulation. 

In [8]:
sj_zip_wages_df['PerCapitaWages'] = sj_zip_wages_df['TotalWages']/sj_zip_wages_df['EstimatedPopulation']
sj_zip_wages_df.head()

,Zipcode,Lat,Long,EstimatedPopulation,TotalWages,PerCapitaWages
0,95110,37.34,-121.90,12621.0,366468568.0,29036.412963
1,95111,37.28,-121.83,43578.0,866020686.0,19872.887374
2,95112,37.34,-121.88,34111.0,891795651.0,26143.931606
3,95113,37.33,-121.89,1049.0,37924110.0,36152.631077
4,95116,37.35,-121.85,35357.0,623888214.0,17645.394519


In [9]:
print("The are", sj_zip_wages_df.shape[0], "ZIP codes\n")
print(list(sj_zip_wages_df['Zipcode']))

The are 28 ZIP codes

[95110, 95111, 95112, 95113, 95116, 95117, 95118, 95119, 95120, 95121, 95122, 95123, 95124, 95125, 95126, 95127, 95128, 95129, 95130, 95131, 95132, 95133, 95134, 95135, 95136, 95138, 95139, 95148]


Import and install relevant packages.

In [10]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import numpy as np # library to handle data in a vectorized manner

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

Draw map of San Jose, CA

In [11]:
address = 'San Jose, CA'

geolocator = Nominatim(user_agent="sj_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Jose, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Jose, CA are 37.3361905, -121.8905833.


In [12]:
# create map of San Jose using latitude and longitude values
map_sj = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(sj_zip_wages_df['Lat'], sj_zip_wages_df['Long'], sj_zip_wages_df['Zipcode'].astype(str)):  
                                                                            #  ^^
                                                                            # need to convert Zipcode to string to display properly 
    
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sj)  
    
map_sj

### 3.2 Foursquare Location Data

In [27]:
import requests

Define Foursquare Credentials and Version

In [28]:
CLIENT_ID = 'IH5AG3QTLSUDQCJ01DLXNF3I4NBHNJ0MHSODAIE1IGQSZS3K' # your Foursquare ID
CLIENT_SECRET = 'UBZ2GRFVDKJVI0YLC13PKS4L2CIMQ1CSX20Z4BOY5CYQ2KZD' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)




Your credentails:
CLIENT_ID: IH5AG3QTLSUDQCJ01DLXNF3I4NBHNJ0MHSODAIE1IGQSZS3K
CLIENT_SECRET:UBZ2GRFVDKJVI0YLC13PKS4L2CIMQ1CSX20Z4BOY5CYQ2KZD


In [29]:
search_query = 'Acupuncture'
radius = 16000
LIMIT = 500

In [30]:
url_acu = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)

#url_acu1 = url_acu + "&offset=50"
url_acu

'https://api.foursquare.com/v2/venues/search?client_id=IH5AG3QTLSUDQCJ01DLXNF3I4NBHNJ0MHSODAIE1IGQSZS3K&client_secret=UBZ2GRFVDKJVI0YLC13PKS4L2CIMQ1CSX20Z4BOY5CYQ2KZD&ll=37.3361905,-121.8905833&v=20190605&query=Acupuncture&radius=16000&limit=500'

In [31]:
results = requests.get(url_acu).json()

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
#print(venues)

# tranform venues into a dataframe
sj_acu_df = pd.json_normalize(venues)
sj_acu_df.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4ce1a79ef8a4a1438ddeebbc,Nurture Acupuncture,"[{'id': '52e81612bcbc57f1066b7a3b', 'name': 'A...",v-1594014279,False,1520 The Alameda #130,37.335472,-121.915164,"[{'label': 'display', 'lat': 37.33547199999999...",2177,95126,US,San Jose,CA,United States,"[1520 The Alameda #130, San Jose, CA 95126, Un...",NaN,NaN
1,58e178060e0a1e6dd8a4f3e3,Charles Lin Acupuncture Clinic,"[{'id': '52e81612bcbc57f1066b7a3b', 'name': 'A...",v-1594014279,False,475 N 1st St Ste 200,37.343643,-121.896553,"[{'label': 'display', 'lat': 37.34364318847656...",983,95112,US,San Jose,CA,United States,"[475 N 1st St Ste 200, San Jose, CA 95112, Uni...",NaN,NaN
2,51cb8d6d498e66b4ad36c053,Acupuncture Orthopedics & Natural Healing Center,"[{'id': '52e81612bcbc57f1066b7a3b', 'name': 'A...",v-1594014279,False,259 Meridian Ave Ste 8,37.324388,-121.914624,"[{'label': 'display', 'lat': 37.324388, 'lng':...",2500,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 8, San Jose, CA 95126, U...",NaN,NaN
3,58f1732e780eee3965948df0,Numo Acupuncture,"[{'id': '52e81612bcbc57f1066b7a3b', 'name': 'A...",v-1594014279,False,1630 Oakland Rd Ste A110,37.381672,-121.894552,"[{'label': 'display', 'lat': 37.3816716, 'lng'...",5075,95131,US,San Jose,CA,United States,"[1630 Oakland Rd Ste A110, San Jose, CA 95131,...",NaN,NaN
4,51142233e4b07ef2ec1fd677,1-2-3 Acupuncture Clinic (Santa Clara),"[{'id': '52e81612bcbc57f1066b7a3b', 'name': 'A...",v-1594014279,False,3700 Thomas Rd Ste 215,37.386284,-121.960762,"[{'label': 'display', 'lat': 37.386284, 'lng':...",8345,95054,US,Santa Clara,CA,United States,"[3700 Thomas Rd Ste 215 (San Thomas EXP), Sant...",San Thomas EXP,NaN


In [33]:
sj_acu_df.shape

(50, 18)

In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in sj_acu_df.columns if col.startswith('location.')] + ['id']
sj_acu_df_filtered = sj_acu_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
sj_acu_df_filtered['categories'] = sj_acu_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
sj_acu_df_filtered.columns = [column.split('.')[-1] for column in sj_acu_df_filtered.columns]

sj_acu_df_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Nurture Acupuncture,Acupuncturist,1520 The Alameda #130,37.335472,-121.915164,"[{'label': 'display', 'lat': 37.33547199999999...",2177,95126,US,San Jose,CA,United States,"[1520 The Alameda #130, San Jose, CA 95126, Un...",NaN,4ce1a79ef8a4a1438ddeebbc
1,Charles Lin Acupuncture Clinic,Acupuncturist,475 N 1st St Ste 200,37.343643,-121.896553,"[{'label': 'display', 'lat': 37.34364318847656...",983,95112,US,San Jose,CA,United States,"[475 N 1st St Ste 200, San Jose, CA 95112, Uni...",NaN,58e178060e0a1e6dd8a4f3e3
2,Acupuncture Orthopedics & Natural Healing Center,Acupuncturist,259 Meridian Ave Ste 8,37.324388,-121.914624,"[{'label': 'display', 'lat': 37.324388, 'lng':...",2500,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 8, San Jose, CA 95126, U...",NaN,51cb8d6d498e66b4ad36c053
3,Numo Acupuncture,Acupuncturist,1630 Oakland Rd Ste A110,37.381672,-121.894552,"[{'label': 'display', 'lat': 37.3816716, 'lng'...",5075,95131,US,San Jose,CA,United States,"[1630 Oakland Rd Ste A110, San Jose, CA 95131,...",NaN,58f1732e780eee3965948df0
4,1-2-3 Acupuncture Clinic (Santa Clara),Acupuncturist,3700 Thomas Rd Ste 215,37.386284,-121.960762,"[{'label': 'display', 'lat': 37.386284, 'lng':...",8345,95054,US,Santa Clara,CA,United States,"[3700 Thomas Rd Ste 215 (San Thomas EXP), Sant...",San Thomas EXP,51142233e4b07ef2ec1fd677


Keep only venues that are in San Jose

In [35]:
sj_acu_df_filtered = sj_acu_df_filtered[sj_acu_df_filtered.city == 'San Jose']
sj_acu_df_filtered.reset_index(drop = True, inplace = True)
sj_acu_df_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Nurture Acupuncture,Acupuncturist,1520 The Alameda #130,37.335472,-121.915164,"[{'label': 'display', 'lat': 37.33547199999999...",2177,95126,US,San Jose,CA,United States,"[1520 The Alameda #130, San Jose, CA 95126, Un...",NaN,4ce1a79ef8a4a1438ddeebbc
1,Charles Lin Acupuncture Clinic,Acupuncturist,475 N 1st St Ste 200,37.343643,-121.896553,"[{'label': 'display', 'lat': 37.34364318847656...",983,95112,US,San Jose,CA,United States,"[475 N 1st St Ste 200, San Jose, CA 95112, Uni...",NaN,58e178060e0a1e6dd8a4f3e3
2,Acupuncture Orthopedics & Natural Healing Center,Acupuncturist,259 Meridian Ave Ste 8,37.324388,-121.914624,"[{'label': 'display', 'lat': 37.324388, 'lng':...",2500,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 8, San Jose, CA 95126, U...",NaN,51cb8d6d498e66b4ad36c053
3,Numo Acupuncture,Acupuncturist,1630 Oakland Rd Ste A110,37.381672,-121.894552,"[{'label': 'display', 'lat': 37.3816716, 'lng'...",5075,95131,US,San Jose,CA,United States,"[1630 Oakland Rd Ste A110, San Jose, CA 95131,...",NaN,58f1732e780eee3965948df0
4,Acupuncture,Acupuncturist,NaN,37.225533,-121.859755,"[{'label': 'display', 'lat': 37.22553319504755...",12617,95120,US,San Jose,CA,United States,"[San Jose, CA 95120, United States]",NaN,4f385cc8e4b0571dc87a817b
5,Green Acupuncture,Massage Studio,265 Meridian Ave. #9,37.324160,-121.914230,"[{'label': 'display', 'lat': 37.32416, 'lng': ...",2484,95126,US,San Jose,CA,United States,"[265 Meridian Ave. #9, San Jose, CA 95126, Uni...",NaN,5b7a488d791871002c9b8d58
6,Kim Wah Acupuncture,Acupuncturist,1818 Tully Rd,37.322895,-121.824707,"[{'label': 'display', 'lat': 37.32289505004883...",6016,95122,US,San Jose,CA,United States,"[1818 Tully Rd (King Rd), San Jose, CA 95122, ...",King Rd,4c676bfef101ef3bfea376e9
7,Vinh Quang Acupuncture & Chinese Herbs,Acupuncturist,1001 Story RD,37.333230,-121.856727,"[{'label': 'display', 'lat': 37.33323, 'lng': ...",3014,NaN,US,San Jose,CA,United States,"[1001 Story RD, San Jose, CA, United States]",NaN,57faf27e498eb89d33b574ca
8,Lokahi Acupuncture,Acupuncturist,259 Meridian Ave Ste 14,37.324516,-121.913934,"[{'label': 'display', 'lat': 37.32451639011752...",2441,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 14 (at San Carlos), San ...",at San Carlos,4b429fbdf964a520b4d725e3
9,Zhu's Neuro-Acupuncture Center,Acupuncturist,1754 Technology Dr Ste 225,37.369337,-121.919080,"[{'label': 'display', 'lat': 37.369337, 'lng':...",4469,95110,US,San Jose,CA,United States,"[1754 Technology Dr Ste 225, San Jose, CA 9511...",NaN,4fe35b50e4b044e6b9b778a3


There are a few venues with no ZIP codes. Find this information by internet search and fill in the ones that are currently still practicing at the given location. Remove ones who have moved or closed down their business.  

In [36]:
sj_acu_df_filtered.at[7, 'postalCode'] = '95122'
sj_acu_df_filtered.at[30, 'postalCode'] = '95123'
sj_acu_df_filtered.at[33, 'postalCode'] = '95121'

sj_acu_df_filtered.dropna(subset = ['postalCode'], axis = 0, inplace = True)
sj_acu_df_filtered.reset_index(drop = True, inplace = True)
sj_acu_df_filtered


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Nurture Acupuncture,Acupuncturist,1520 The Alameda #130,37.335472,-121.915164,"[{'label': 'display', 'lat': 37.33547199999999...",2177,95126,US,San Jose,CA,United States,"[1520 The Alameda #130, San Jose, CA 95126, Un...",NaN,4ce1a79ef8a4a1438ddeebbc
1,Charles Lin Acupuncture Clinic,Acupuncturist,475 N 1st St Ste 200,37.343643,-121.896553,"[{'label': 'display', 'lat': 37.34364318847656...",983,95112,US,San Jose,CA,United States,"[475 N 1st St Ste 200, San Jose, CA 95112, Uni...",NaN,58e178060e0a1e6dd8a4f3e3
2,Acupuncture Orthopedics & Natural Healing Center,Acupuncturist,259 Meridian Ave Ste 8,37.324388,-121.914624,"[{'label': 'display', 'lat': 37.324388, 'lng':...",2500,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 8, San Jose, CA 95126, U...",NaN,51cb8d6d498e66b4ad36c053
3,Numo Acupuncture,Acupuncturist,1630 Oakland Rd Ste A110,37.381672,-121.894552,"[{'label': 'display', 'lat': 37.3816716, 'lng'...",5075,95131,US,San Jose,CA,United States,"[1630 Oakland Rd Ste A110, San Jose, CA 95131,...",NaN,58f1732e780eee3965948df0
4,Acupuncture,Acupuncturist,NaN,37.225533,-121.859755,"[{'label': 'display', 'lat': 37.22553319504755...",12617,95120,US,San Jose,CA,United States,"[San Jose, CA 95120, United States]",NaN,4f385cc8e4b0571dc87a817b
5,Green Acupuncture,Massage Studio,265 Meridian Ave. #9,37.324160,-121.914230,"[{'label': 'display', 'lat': 37.32416, 'lng': ...",2484,95126,US,San Jose,CA,United States,"[265 Meridian Ave. #9, San Jose, CA 95126, Uni...",NaN,5b7a488d791871002c9b8d58
6,Kim Wah Acupuncture,Acupuncturist,1818 Tully Rd,37.322895,-121.824707,"[{'label': 'display', 'lat': 37.32289505004883...",6016,95122,US,San Jose,CA,United States,"[1818 Tully Rd (King Rd), San Jose, CA 95122, ...",King Rd,4c676bfef101ef3bfea376e9
7,Vinh Quang Acupuncture & Chinese Herbs,Acupuncturist,1001 Story RD,37.333230,-121.856727,"[{'label': 'display', 'lat': 37.33323, 'lng': ...",3014,95122,US,San Jose,CA,United States,"[1001 Story RD, San Jose, CA, United States]",NaN,57faf27e498eb89d33b574ca
8,Lokahi Acupuncture,Acupuncturist,259 Meridian Ave Ste 14,37.324516,-121.913934,"[{'label': 'display', 'lat': 37.32451639011752...",2441,95126,US,San Jose,CA,United States,"[259 Meridian Ave Ste 14 (at San Carlos), San ...",at San Carlos,4b429fbdf964a520b4d725e3
9,Zhu's Neuro-Acupuncture Center,Acupuncturist,1754 Technology Dr Ste 225,37.369337,-121.919080,"[{'label': 'display', 'lat': 37.369337, 'lng':...",4469,95110,US,San Jose,CA,United States,"[1754 Technology Dr Ste 225, San Jose, CA 9511...",NaN,4fe35b50e4b044e6b9b778a3


Calculate the number of venues in each neighborhood (ZIP code)

In [37]:
sj_acu_zip = sj_acu_df_filtered.groupby('postalCode').count()
sj_acu_zip['Zipcode'] = sj_acu_zip.index
sj_acu_zip 

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,id,Zipcode
postalCode,,,,,,,,,,,,,,,
95110,2,2,2,2,2,2,2,2,2,2,2,2,0,2,95110
95112,1,1,1,1,1,1,1,1,1,1,1,1,0,1,95112
95117,1,1,0,1,1,1,1,1,1,1,1,1,0,1,95117
95120,1,1,0,1,1,1,1,1,1,1,1,1,0,1,95120
95121,1,1,0,1,1,1,1,1,1,1,1,1,0,1,95121
95122,2,2,2,2,2,2,2,2,2,2,2,2,1,2,95122
95123,1,1,1,1,1,1,1,1,1,1,1,1,0,1,95123
95125,5,5,4,5,5,5,5,5,5,5,5,5,1,5,95125
95126,4,4,4,4,4,4,4,4,4,4,4,4,1,4,95126


In [38]:
sj_acu_zip = sj_acu_zip[['Zipcode', 'name']]
sj_acu_zip['Zipcode'] = sj_acu_zip['Zipcode'].astype(int)

sj_acu_zip.sort_values(by = 'Zipcode', inplace = True)
sj_acu_zip.columns = ['Zipcode', 'numbers']

#sj_acu_zip.reset_index(drop = True, inplace = True)
sj_acu_zip


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Zipcode,numbers
postalCode,,
95110,95110,2
95112,95112,1
95117,95117,1
95120,95120,1
95121,95121,1
95122,95122,2
95123,95123,1
95125,95125,5
95126,95126,4


Add this data into the existing dataframe

In [39]:
sj_zip_wages_df['NumAcuClinics'] = ""

count = 0

for zipcode in sj_zip_wages_df['Zipcode']:
    
    if zipcode in list(sj_acu_zip['Zipcode']):
       # print(zip, " is in the list")
        
        list_index = list(sj_acu_zip['Zipcode']).index(zipcode)
              
        sj_zip_wages_df.at[count, 'NumAcuClinics'] = sj_acu_zip.iloc[list_index]['numbers']
    else:
        # print(zip, " is not in the list")
        
        sj_zip_wages_df.at[count, 'NumAcuClinics'] = 0
        
    count = count + 1

In [40]:
sj_zip_wages_df

,Zipcode,Lat,Long,EstimatedPopulation,TotalWages,PerCapitaWages,NumAcuClinics
0,95110,37.34,-121.90,12621.0,3.664686e+08,29036.412963,2
1,95111,37.28,-121.83,43578.0,8.660207e+08,19872.887374,0
2,95112,37.34,-121.88,34111.0,8.917957e+08,26143.931606,1
3,95113,37.33,-121.89,1049.0,3.792411e+07,36152.631077,0
4,95116,37.35,-121.85,35357.0,6.238882e+08,17645.394519,0
5,95117,37.31,-121.96,22030.0,6.661914e+08,30240.189696,1
6,95118,37.25,-121.88,26249.0,8.777164e+08,33438.090632,0
7,95119,37.22,-121.78,8171.0,2.887622e+08,35339.883368,0
8,95120,37.19,-121.83,33486.0,1.704123e+09,50890.606821,1
9,95121,37.30,-121.80,30427.0,7.641766e+08,25115.081638,1


### 3.3 Crime Rate, Unemployment Rate, Bachelor Degree Percentage, Median Home Price

Import remaining data from manually tabulated Excel spreadsheet

In [41]:
sj_other_data_df = pd.read_excel('San_Jose_Other_Data.xlsx')
sj_other_data_df.head()

,Zipcode,Crime_rate,Unemployment_Rate_25,Bachelor_Degree_Percentage,Median_Home_Zillow
0,95110,91,4.9,32.4,820
1,95111,32,6.1,19.6,774
2,95112,85,6.2,35.0,849
3,95113,86,4.5,72.8,763
4,95116,37,6.9,16.6,712


### 3.4 Final Data Aggregation

Combine data from all the previous sections. Remove TotalWages and keep only PerCapitaWages

In [42]:
sj_zip_merged_df = sj_zip_wages_df
sj_zip_merged_df = sj_zip_merged_df.join(sj_other_data_df.set_index('Zipcode'), on='Zipcode')
sj_zip_merged_df.columns = ['Zipcode', 'Lat','Long','EstPop', 'TotalWages', 'PerCapitaWages', 'Clinics', 'CrimeRate', 'UnemployRate', 
                           'BSPercent', 'HomePrice']

sj_zip_merged_df.drop(['TotalWages'], axis = 1, inplace = True)
sj_zip_merged_df.head()


,Zipcode,Lat,Long,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
0,95110,37.34,-121.90,12621.0,29036.412963,2,91,4.9,32.4,820
1,95111,37.28,-121.83,43578.0,19872.887374,0,32,6.1,19.6,774
2,95112,37.34,-121.88,34111.0,26143.931606,1,85,6.2,35.0,849
3,95113,37.33,-121.89,1049.0,36152.631077,0,86,4.5,72.8,763
4,95116,37.35,-121.85,35357.0,17645.394519,0,37,6.9,16.6,712


### 3.4 Clustering Modeling

Transform data by normalizing the columns with the StandardScaler. 

In [43]:
#from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

X = sj_zip_merged_df[['EstPop', 'PerCapitaWages', 'Clinics', 'CrimeRate', 'UnemployRate', 'BSPercent', 'HomePrice']]
#sj_zip_transformed = MinMaxScaler().fit_transform(X)
sj_zip_transformed = StandardScaler().fit_transform(X)




/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


Run k-means clustering algorithm

In [44]:
# set number of clusters
kclusters = 5

# run k-means clustering
#kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sj_zip_transformed)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sj_zip_transformed)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 

print("Cluster Labels:")
print(kmeans.labels_)


Cluster Labels:
[1 4 1 2 4 1 1 2 3 4 4 1 1 0 0 4 0 3 3 0 1 4 2 2 1 3 1 1]


## 4. Results
<a id = "results"></a>

Insert resulting cluster labels back into merged dataframe

In [45]:
sj_zip_merged_df.insert(1, 'Cluster Labels', kmeans.labels_)

# Uncomment following code if trying to re-run this individual code. 
# As cannot reinsert existing column. 
# sj_zip_merged_df['Cluster Labels'] = kmeans.labels_

sj_zip_merged_df.head()


,Zipcode,Cluster Labels,Lat,Long,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
0,95110,1,37.34,-121.90,12621.0,29036.412963,2,91,4.9,32.4,820
1,95111,4,37.28,-121.83,43578.0,19872.887374,0,32,6.1,19.6,774
2,95112,1,37.34,-121.88,34111.0,26143.931606,1,85,6.2,35.0,849
3,95113,2,37.33,-121.89,1049.0,36152.631077,0,86,4.5,72.8,763
4,95116,4,37.35,-121.85,35357.0,17645.394519,0,37,6.9,16.6,712


Re-draw San Jose map with color-code clustering labels

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sj_zip_merged_df['Lat'], sj_zip_merged_df['Long'], sj_zip_merged_df['Zipcode'], sj_zip_merged_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's begin with Cluster 1, which has the most neighborhoods

In [47]:
sj_zip_merged_df.loc[sj_zip_merged_df['Cluster Labels'] == 1, 
                   sj_zip_merged_df.columns[[0] + list(range(4, sj_zip_merged_df.shape[1]))]]

,Zipcode,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
0,95110,12621.0,29036.412963,2,91,4.9,32.4,820
2,95112,34111.0,26143.931606,1,85,6.2,35.0,849
5,95117,22030.0,30240.189696,1,21,5.5,45.5,1275
6,95118,26249.0,33438.090632,0,65,4.3,46.6,1128
11,95123,50481.0,32382.124086,1,67,4.0,41.1,946
12,95124,39234.0,38621.629938,0,69,4.4,53.0,1275
20,95132,34344.0,30705.994118,0,65,5.9,45.7,1151
24,95136,35078.0,33568.082074,0,68,4.7,46.7,961
26,95139,5634.0,36503.131523,0,40,6.2,47.7,904
27,95148,37541.0,30403.984470,0,61,5.7,39.0,1026


In this cluster, it appears that all the parameters such as per capita wages, unemployment, bachelor degree percentage, and home prices, are average, neither too high nor too low. The number of clinics here are also very sparse. 

Let's continue to examine Cluster 4, which has the 2nd most neighborhoods.

In [48]:
sj_zip_merged_df.loc[sj_zip_merged_df['Cluster Labels'] == 4, 
                   sj_zip_merged_df.columns[[0] + list(range(4, sj_zip_merged_df.shape[1]))]]

,Zipcode,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
1,95111,43578.0,19872.887374,0,32,6.1,19.6,774
4,95116,35357.0,17645.394519,0,37,6.9,16.6,712
9,95121,30427.0,25115.081638,1,59,6.0,29.6,852
10,95122,41936.0,16719.609953,2,29,6.4,15.1,726
15,95127,46641.0,22820.174160,0,45,5.5,23.4,792
21,95133,20337.0,26582.617544,0,24,7.3,35.3,870


Cluster 4 does not appear to be wealthy, with lower wages and home prices. The unemployment rate is higher, and the bachelor degree percentage is low. The crime rate is low, a result that may run a bit counterintuitive. There is not a lot of existing competition, as the number of clinics is sparse as well. 

Let's examine the remaining clusters, which have 4 neighborhoods each. We will start with Cluster 0

In [49]:
sj_zip_merged_df.loc[sj_zip_merged_df['Cluster Labels'] == 0, 
                   sj_zip_merged_df.columns[[0] + list(range(4, sj_zip_merged_df.shape[1]))]]

,Zipcode,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
13,95125,41048.0,43425.502022,5,92,4.8,54.0,1331
14,95126,23076.0,36464.311362,4,93,4.8,52.0,1025
16,95128,25327.0,33020.557231,9,80,3.6,43.8,1159
19,95131,24403.0,37677.994919,6,55,4.4,55.9,1077


This is an affluent cluster, with the high per capita wages and home prices. The unemployment rate is also low, as expected. The crime rate is rather high for this area. However, the biggest observation is the stiff competition, with a large number of existing clinics. 

Next We will examine Cluster 2.

In [50]:
sj_zip_merged_df.loc[sj_zip_merged_df['Cluster Labels'] == 2, 
                   sj_zip_merged_df.columns[[0] + list(range(4, sj_zip_merged_df.shape[1]))]]

,Zipcode,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
3,95113,1049.0,36152.631077,0,86,4.5,72.8,763
7,95119,8171.0,35339.883368,0,128,4.2,41.5,953
22,95134,12670.0,51631.955722,0,92,3.0,75.8,951
23,95135,17221.0,42079.977063,0,121,4.7,61.3,1169


This cluster is wealthy and highly educated, with low unemployment rates. The home prices are not as high, which may indicate an area that is still growing. There is also no competition in this area either. However, the recorded crime rate is high. 

Lastly, let's look at Cluster 3

In [51]:
sj_zip_merged_df.loc[sj_zip_merged_df['Cluster Labels'] == 3, 
                   sj_zip_merged_df.columns[[0] + list(range(4, sj_zip_merged_df.shape[1]))]]

,Zipcode,EstPop,PerCapitaWages,Clinics,CrimeRate,UnemployRate,BSPercent,HomePrice
8,95120,33486.0,50890.606821,1,52,3.1,71.3,1508
17,95129,32839.0,41750.644569,0,53,3.7,72.9,1751
18,95130,10841.0,36396.430311,0,64,3.7,55.7,1419
25,95138,15421.0,57789.241554,0,42,5.3,59.2,1213


This cluster appears to be the wealthiest, both in terms of wages and home prices. This cluster is also highly educated with high bachelor degree percentages, along with low unemployment rates and low crime rates. The existing competition is also very weak, with only 1 clinic in the entire cluster.

## 5. Discussion
<a id = "discussion"></a>

From the results of the previous section, it is reasonable to avoid setting up a clinic in Cluster 0. The biggest drawback is existing stiff competition, and the high saturation can be a challenge getting the business off the ground, in addition to future business growth. 

Another cluster that I would advise against would be Cluster 4. Despite weak competition and low crime rate, the neighborhoods are not wealthy, with higher unemployment rates. This is not an ideal location to place your business in. 

As for Cluster 1, whose attributes are average, these neighborhoods probably would be considered a "safe bet" and recommended over Clusters 0 and 4. 

However, it is also possible to do better by examining the results of Clusters 2 and 3. Both are affluent and highly educated, with weak competition. The downside to Cluster 2 is the high recorded crime rate. The lower home prices may or may not be a reflection of that. It can also indicate that it is a neighborhood currently under growth. 

Based on the findings of the data, we can conclude that Cluster 3 contains the neighborhoods with conditions most favorable to start an acupuncture clinic in.

## 6. Conclusion
<a id = "conclusion"></a>

This project provided me an opportunity to compile, analyze, and process demographic data of real neighborhoods. The work was done under the context of finding a suitable business location for an acupuncture clinic. The parameters I inspected was population, per capita wages, acupuncture clinics in the neighborhood, crime rate, unemployment rate (over 25), bachelor degree percentage (over 25), and median home price. 

This data was further analyzed using the k-means clustering algorithm, where its results helped provide insight on which neighborhoods would be most ideal to setup an acupuncture practice.      

There are a few other things worth pointing out about the model's findings that could serve as future directions. The first is that average commercial rent prices were not included in the model, which may be an important deciding factor. For example, a wealthier neighborhood may also charge higher commercial rent, which can be a deterrent for setting up a business there. The second is that Foursquare location data only returns 50 results for a venue search. Based on personal experience residing in this area, the number of acupuncture clinics reported is lower than expected. This factor affects the accuracy of the model. Lastly, using the total crime rate may not give the entire picture either. Violent crimes tend to discourage starting a business in the area, while the impact of lesser crimes can be much lower. 

Despite this, the findings of this project are valuable and provide a good starting point for further analysis.   